In [ ]:
!pip install scrapy
!pip install spider3

In [ ]:
import json
import re
import pandas as pd
import scrapy
from pandas.testing import assert_frame_equal # to compare two dataframes
# YOUR CODE HERE (OPTION) 
# Nếu cần các thư viện khác thì bạn có thể import thêm tại đây
import datetime

In [ ]:
!scrapy startproject fifa_crawler

Error: Module 'fifa_crawler' already exists


In [ ]:
cd fifa_crawler/fifa_crawler

[Errno 2] No such file or directory: 'fifa_crawler/fifa_crawler'
/content/fifa_crawler/fifa_crawler


In [ ]:
!ls
!touch spiders/collect_teams_urls.py

__init__.py  middlewares.py  __pycache__  spiders
items.py     pipelines.py    settings.py


In [27]:
import scrapy

class collect_team_url(scrapy.Spider):
  name='teams_urls' 
  
  def start_requests(self):
    urls = ['https://sofifa.com/teams?col=oa&sort=desc&offset=0']
    
    # YOUR CODE HERE
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    for i in range(1,13):
        url = f'https://sofifa.com/teams?col=oa&sort=desc&offset={i*60}' 
        urls.append(url)

    for url in urls:
      yield scrapy.Request(url=url, headers=headers, callback=self.parse)

  def parse(self, response):
    team_urls = response.css('td.col-name-wide a::attr(href)').extract()
    team_ids = []
    for url in team_urls:
      if url.split('/')[1] == "team":
        team_id = url.split('/')[2]
        team_url = f"/team/{team_id}"
        team_item = {"team_url": team_url}
        yield team_item

In [46]:
!scrapy crawl teams_urls -o dataset/teams_urls.json

2023-04-14 16:53:09 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: fifa_crawler)
2023-04-14 16:53:09 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.9.16 (main, Dec  7 2022, 01:11:51) - [GCC 9.4.0], pyOpenSSL 23.1.1 (OpenSSL 3.1.0 14 Mar 2023), cryptography 40.0.1, Platform Linux-5.10.147+-x86_64-with-glibc2.31
2023-04-14 16:53:09 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'fifa_crawler',
 'FEED_EXPORT_ENCODING': 'utf-8',
 'NEWSPIDER_MODULE': 'fifa_crawler.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['fifa_crawler.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2023-04-14 16:53:09 [asyncio] DEBUG: Using selector: EpollSelector
2023-04-14 16:53:09 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2023-04-14 16:53:09 [scrapy.utils.log] DEBUG

In [78]:
df_test = pd.read_json('/content/fifa_crawler/fifa_crawler/dataset/teams_info.json', encoding='utf-8-sig')
df_test.iloc[0]

Name                                Manchester City
League                     [England] Premier League
Overall                                          85
Attack                                           85
Midfield                                         87
Defence                                          83
Home stadium                         Etihad Stadium
Rival team                        Manchester United
International prestige                           10
Domestic prestige                              10.0
Club worth                                       \n
Starting XI average age                       27.64
Whole team average age                        25.27
Short free kick                        K. De Bruyne
Name: 0, dtype: object

In [74]:
import scrapy
import json
import re
import datetime

class collect_team_info(scrapy.Spider):
  name='teams_info'
  
  def __init__(self):
    try:
      with open('dataset/teams_urls.json') as f:
        self.teams = json.load(f)
      self.team_count = 1
    except IOError:
      print("File not found")

  def start_requests(self):
    # YOUR CODE HERE
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    for team in self.teams:
      url = 'https://sofifa.com' + team['team_url'] + '?units=mks'
      yield scrapy.Request(url=url, headers=headers, callback=self.parse)
  
  def parse(self, response):
      # YOUR CODE HERE
      team_info = {}

      team_info["Name"] = response.css('div.info h1::text').get();

      team_info["League"] = response.css('div.meta.ellipsis a::text').get()

      ratings = response.css('section.card .bp3-tag.p::text').getall()

      team_info["Overall"] = ratings[0]

      team_info["Attack"] = ratings[1]
      
      team_info["Midfield"] = ratings[2]
      
      team_info["Defence"] = ratings[3]

      team_info["Home stadium"] = response.css('ul.pl li.ellipsis:contains("Home stadium")::text').get()
      
      team_info["Rival team"] = response.css('ul.pl li:contains("Rival team") a::text').get()

      team_info["International prestige"] = response.css('ul.pl li:contains("International prestige") span::text').get()

      team_info["Domestic prestige"] = response.css('ul.pl li:contains("Domestic prestige") span::text').get()

      team_info["Club worth"] = response.css('ul.pl li:contains("Club worth")::text').get()

      team_info["Starting XI average age"] = response.css('ul.pl li:contains("Starting XI average age")::text').get()

      team_info["Whole team average age"] = response.css('ul.pl li:contains("Whole team average age")::text').get()

      team_info["Captain"] = response.css('ul.pl li:contains("Captain") a::text').get()

      team_info["Short free kick"] = response.css('ul.pl li:contains("Short free kick") a::text').get()

      team_info["Long free kick"] = response.css('ul.pl li:contains("Long free kick") a::text').get()

      team_info["Left short free kick"] = response.css('ul.pl li:contains("Left short free kick") a::text').get()
      
      team_info["Right short free kick"] = response.css('ul.pl li:contains("Right short free kick") a::text').get()
      
      team_info["Penalties"] = response.css('ul.pl li:contains("Penalties") a::text').get()
      
      team_info["Left corner"] = response.css('ul.pl li:contains("Left corner") a::text').get()
      
      team_info["Right corner"] = response.css('ul.pl li:contains("Right corner") a::text').get()

      yield team_info

Sau khi đã hoàn thành class ở trên và lưu lại trong file <b>collect_players_info.py</b>, các bạn sẽ tiếp tục chạy câu lệnh bên dưới để thu thập thông tin chi tiết của toàn bộ 720 cầu thủ và xuất ra file <b>players_info.json</b> ở cùng đường dẫn dataset như trên.

In [ ]:
!rm /content/fifa_crawler/fifa_crawler/dataset/teams_info.json
!scrapy crawl teams_info -o dataset/teams_info.json